In [11]:
# Now let's implement BERT with the correct column names
import torch
import numpy as np
import pandas as pd

from extended_function import *
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder

d:\School Documnets\Internship 2\Employee_onboarding_chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
df = pd.read_csv("../data/combine_df.csv")

In [13]:
# Initialize tokenizer and prepare data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare the data with correct column names
messages = df['Employee_message'].apply(clean_message)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['service'])

In [14]:
# Tokenize with smaller max_length
max_length = 64
encodings = tokenizer(
    messages.tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='pt'
)

In [15]:
# Create dataset
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']
labels_tensor = torch.tensor(labels)
dataset = TensorDataset(input_ids, attention_mask, labels_tensor)

# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create dataloaders with smaller batch size
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
# Initialize model
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

print("Starting training...")

# Training loop with early stopping
model.train()
patience = 2
best_loss = float('inf')
patience_counter = 0
num_epochs = 25

for epoch in range(num_epochs):  
    epoch_loss = 0
    
    for i, batch in enumerate(train_dataloader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device).long()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # Print every 10 batches
        if (i + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {i+1}/{len(train_dataloader)}, Loss: {loss.item():.4f}")
    
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} average loss: {avg_epoch_loss:.4f}")
    
    # Early stopping check
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

print("\nTraining complete.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...
Epoch 1, Batch 10/55, Loss: 2.3675
Epoch 1, Batch 20/55, Loss: 2.2729
Epoch 1, Batch 30/55, Loss: 2.2311
Epoch 1, Batch 40/55, Loss: 2.2491
Epoch 1, Batch 50/55, Loss: 2.0678
Epoch 1 average loss: 2.2046
Epoch 2, Batch 10/55, Loss: 2.0310
Epoch 2, Batch 20/55, Loss: 2.0646
Epoch 2, Batch 30/55, Loss: 1.8512
Epoch 2, Batch 40/55, Loss: 1.8567
Epoch 2, Batch 50/55, Loss: 1.8521
Epoch 2 average loss: 1.9443
Epoch 3, Batch 10/55, Loss: 1.6054
Epoch 3, Batch 20/55, Loss: 1.6912
Epoch 3, Batch 30/55, Loss: 1.6055
Epoch 3, Batch 40/55, Loss: 1.5910
Epoch 3, Batch 50/55, Loss: 1.4940
Epoch 3 average loss: 1.5590
Epoch 4, Batch 10/55, Loss: 1.0497
Epoch 4, Batch 20/55, Loss: 1.3648
Epoch 4, Batch 30/55, Loss: 0.9909
Epoch 4, Batch 40/55, Loss: 1.0097
Epoch 4, Batch 50/55, Loss: 0.8394
Epoch 4 average loss: 1.1207
Epoch 5, Batch 10/55, Loss: 0.7479
Epoch 5, Batch 20/55, Loss: 0.7670
Epoch 5, Batch 30/55, Loss: 0.9787
Epoch 5, Batch 40/55, Loss: 1.0514
Epoch 5, Batch 50/55, L

In [ ]:
# Save the whole model
print("\nSaving model...")
torch.save(model, "../model/model_25epochs.pth")

# Save the model's state dictionary
torch.save(model.state_dict(), "../model/model_state_25epochs.pth")

# Assuming 'model' is your model and 'optimizer' is your optimizer
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}
# torch.save(checkpoint, '../model/checkpoint.pth')

print("\nSaving complete.")


Saving model...

Saving complete.


In [27]:
# Evaluation
print("\nEvaluating model...")
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = np.mean(np.array(predictions) == np.array(true_labels))
print(f"Test Accuracy: {accuracy:.4f}")


Evaluating model...
Test Accuracy: 0.9545


In [28]:
# Load saved model
print("\nLoading model...")
model = torch.load("../model/model_25epochs.pth", weights_only=False)
print("\nModel Load Completed")


Loading model...

Model Load Completed


In [29]:
# Function to predict intent using the BERT model
def predict_intent_bert(text):
    model.eval()
    encoding = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        predicted_label = label_encoder.inverse_transform([predicted.cpu().item()])[0]
        
    return predicted_label, confidence.cpu().item()

In [30]:
# Test the model with sample messages
test_messages = [
    "I need to request time off for next week",
    "My computer keeps crashing and I can't work",
    "I want to set up training for my team",
    "I need access to the sales database",
    "I want to report the harm",
    "I would like to sign up for a training session on effective communication.",
    "Could you please do my performance review meeting?",
    "I need access to the shared project drive; can you help me out?",
    "I’m planning to relocate to the New York office; what is the process?",
    "I want to report a safety incident that occurred in the warehouse.",
    "I’d like to request time off for next month due to a personal commitment.",
    "Can you help me enroll in the company benefits program, specifically health insurance?",
    "I need to report an incident of harassment that I witnessed at work.",
    "How do I set my performance goals for the next quarter?",
    # "My computer isn’t booting up properly; can you assist me with this issue?"
]

print("\nTesting model with sample messages:")
print("-" * 50)
for message in test_messages:
    intent, confidence = predict_intent_bert(message)
    print(f"Message: {message}")
    print(f"Predicted Intent: {intent}")
    print(f"Confidence: {confidence:.2f}")
    print("-" * 50)


Testing model with sample messages:
--------------------------------------------------
Message: I need to request time off for next week
Predicted Intent: time_off_report
Confidence: 0.99
--------------------------------------------------
Message: My computer keeps crashing and I can't work
Predicted Intent: it_issue_report
Confidence: 0.97
--------------------------------------------------
Message: I want to set up training for my team
Predicted Intent: training_request
Confidence: 0.63
--------------------------------------------------
Message: I need access to the sales database
Predicted Intent: access_request
Confidence: 0.99
--------------------------------------------------
Message: I want to report the harm
Predicted Intent: harassment_report
Confidence: 0.92
--------------------------------------------------
Message: I would like to sign up for a training session on effective communication.
Predicted Intent: training_request
Confidence: 0.92
----------------------------------